In [1]:
#import the Dash package
from dash import Dash, html, dcc, Input, Output
from jupyter_dash import JupyterDash

import warnings
warnings.filterwarnings('ignore')

In [2]:
import data_processor
import plot_maker
from constants import DESCRIPTION_URL_DICTIONARY
import pandas as pd 
from datetime import datetime, date, timedelta

from dash import Dash, dcc, html, Input, Output, no_update

data = data_processor.DataProcessor().get_data()
app = JupyterDash(__name__)

app.layout = html.Div([dcc.DatePickerSingle(id='date-picker-single',
min_date_allowed=date(2012, 10, 1),
max_date_allowed=date(2017, 10, 30),
initial_visible_month=date(2015, 8, 5),
date=date(2015, 8, 25)
    ),
    html.Br(),
    html.Br(),
    dcc.Slider(0, 23, 1, value=1, id='time-select'),
    dcc.Graph(id='plot', clear_on_unhover=True),
    dcc.Tooltip(id="graph-tooltip")
])


@app.callback(
       Output('plot', 'figure'),
       Input('date-picker-single', 'date'),
       Input('time-select', 'value'),
)
def update_figure(date,time):
    datetime_datetime = datetime.fromisoformat(date) + timedelta(hours=time)
    datetime_pandas_timestamp = pd.to_datetime(datetime_datetime)
    fig = plot_maker.PlotMaker(datetime_pandas_timestamp,data).plot_maker() 
    fig.update_traces(hoverinfo="none", hovertemplate=None)
    return fig


@app.callback(
        Output("graph-tooltip", "show"),
        Output("graph-tooltip", "bbox"),
        Output("graph-tooltip", "children"),
        Input('plot', "hoverData"),
        Input('date-picker-single', 'date'),
        Input('time-select', 'value'),
)
def update_hover(hoverData,date,time): 
    
    if hoverData is None:
        return False, no_update, no_update
    pt_lat = hoverData["points"][0]['lat']
    pt_lon = hoverData["points"][0]['lon']
    bbox = hoverData["points"][0]["bbox"]
    
    datetime_datetime = datetime.fromisoformat(date) + timedelta(hours=time)
    datetime_pandas_timestamp = pd.to_datetime(datetime_datetime)

    
    name = data['locations'][(data['locations'].Latitude == pt_lat) |(data['locations'].Longitude == pt_lon)].City.values[0]
    temp = data['temp_in_F'][data['temp_in_F'].datetime == datetime_pandas_timestamp][name].values[0]
    desc = data['weather_description'][data['weather_description'].datetime == datetime_pandas_timestamp][name].values[0]
    img_src = DESCRIPTION_URL_DICTIONARY[desc]
    if len(desc) > 300:
        desc = desc[:100] + '...'

    children = [
        html.Div([
            html.Img(src=img_src, style={"width": "100%"}),
            html.H2(f"{name}", style={"color": "darkblue", "overflow-wrap": "break-word"}),
            html.P(f"{temp:.1f}"u'\N{DEGREE SIGN}F'),
            html.P(f'{desc}'),
        ], style={'width': '200px', 'white-space': 'normal'})
    ]

    return True, bbox, children

app.run_server(mode='inline', port = 8062)

Dash is running on http://127.0.0.1:8062/

